<img width="8%" alt="GitHub.png" src="https://raw.githubusercontent.com/jupyter-naas/awesome-notebooks/master/.github/assets/logos/GitHub.png" style="border-radius: 15%">

# GitHub - Rank interactions by contributors
<a href="https://bit.ly/3JyWIk6">Give Feedback</a> | <a href="https://github.com/jupyter-naas/awesome-notebooks/issues/new?assignees=&labels=bug&template=bug_report.md&title=GitHub+-+Rank+interactions+by+contributors:+Error+short+description">Bug report</a>

**Tags:** #github #reaction #comment #creator #assignees #automation

**Author:** [Varsha Kumar](https://www.linkedin.com/in/varsha-kumar-590466305/)

**Last update:** 2024-06-17 (Created: 2024-06-04)

**Description:** This notebook ranks all interactions made by contributors in a repository.

## Input

In [1]:
import requests
import re
import pandas as pd
import naas_python as naas
from urllib.parse import urlparse
import time
import plotly.express as px
from naas_drivers import github
import ipywidgets as widgets
from IPython.display import display
from IPython.display import display, clear_output

### Setup variables
- `github_token`: personal token creates
- `github_url`: link to the chosen github repo
- `input_csv`: excel file

In [2]:
github_token = naas.secret.get("GITHUB_TOKEN").value
repo_url = "https://github.com/jupyter-naas/workspace"
input_csv = "jupyter-naas_workspace_interactions.csv"

## Model

### Reads excel file

In [3]:
df = pd.read_csv(input_csv)

### Ranks contributors by interactions

In [4]:
def get_interactions(df):
    # Groupby and count
    df = df.groupby(["USER_LOGIN"], as_index=False).agg({"ID": "count"})

    # Cleaning
    df = df.rename(columns={"ID": "NUMBER_OF_INTERACTIONS"})
    return df.sort_values(by="NUMBER_OF_INTERACTIONS", ascending=False).reset_index(drop=True)


df_statistics = get_interactions(df)
pd.DataFrame(df_statistics)

### Create barchart for rankings

In [5]:
def create_ranking_barchart(df, url):
    # Get repository
    repository = url.split("/")[-1]

    # Sort df
    df = df.sort_values(by="NUMBER_OF_INTERACTIONS")

    # Calc commits
    interactions = df.NUMBER_OF_INTERACTIONS.sum()

    # Create fig
    fig = px.bar(
        df,
        y="USER_LOGIN",
        x="NUMBER_OF_INTERACTIONS",
        orientation="h",
        title=f"GitHub - {repository} : Issue interactions by user <br><span style='font-size: 13px;'>Total interactions: {interactions}</span>",
        text="NUMBER_OF_INTERACTIONS",
        labels={"USER_LOGIN": "Login", "NUMBER_OF_INTERACTIONS": "Number of Interactions"},
    )
    fig.update_traces(marker_color="black")
    fig.update_layout(
        plot_bgcolor="#ffffff",
        width=1000,
        height=1200,
        font=dict(family="Arial", size=14, color="black"),
        paper_bgcolor="white",
        xaxis_title=None,
        xaxis_showticklabels=False,
        yaxis_title=None,
        margin_pad=10,
    )
    fig.show()
    return fig

### Create barchart for selected user

In [6]:
def create_selected_user_barchart(df, user, url):
    
    repository = url.split("/")[-1]
    
    # Sort df
    df = df.sort_values(by="AMOUNT")

    # Calc commits
    interactions = df.AMOUNT.sum()

    # Create fig
    fig = px.bar(
        df,
        y="TYPE",
        x="AMOUNT",
        orientation="h",
        title=f"GitHub - {repository} : Issue interactions by user <br><span style='font-size: 13px;'>Total interactions: {interactions}<br>User: {user}</span>",
        text="AMOUNT",
        labels={"TYPE": "type", "AMOUNT": "amount"},
    )
    fig.update_traces(marker_color="black")
    fig.update_layout(
        plot_bgcolor="#ffffff",
        width=1000,
        height=800,
        font=dict(family="Arial", size=14, color="black"),
        paper_bgcolor="white",
        xaxis_title=None,
        xaxis_showticklabels=False,
        yaxis_title=None,
        margin_pad=10,
    )
    fig.show()
    return fig

## Output

### Display rankings on bar chart

In [7]:
fig = create_ranking_barchart(df_statistics, repo_url)

### Display dropdown menu with selected user data

In [8]:
user_dropdown = widgets.Dropdown(
    options=df_statistics['USER_LOGIN'].unique(),
    description='User:',
    disabled=False,
)

def Dropdown_Menu(value):

    x = list(df_statistics['USER_LOGIN'].unique())

    pos = x.index(value)
    user = x[pos]

    user_creations = ((df['USER_LOGIN'] == user) & (df['INTERACTION_TYPE'] == 'CREATOR')).sum()

    user_assignees = ((df['USER_LOGIN'] == user) & (df['INTERACTION_TYPE'] == 'ASSIGNEE')).sum()

    user_comments = ((df['USER_LOGIN'] == user) & (df['INTERACTION_TYPE'] == 'COMMENT')).sum()

    user_reactions = ((df['USER_LOGIN'] == user) & (df['INTERACTION_TYPE'] == 'REACTION')).sum()

    user_commits = ((df['USER_LOGIN'] == user) & (df['INTERACTION_TYPE'] == 'COMMIT')).sum()

    user_data = []
    user_data.append({
        "TYPE": "Creations",
        "AMOUNT": user_creations
    })

    user_data.append({
        "TYPE": "Assignments",
        "AMOUNT": user_assignees
    })

    user_data.append({
        "TYPE": "Comments",
        "AMOUNT": user_comments
    })

    user_data.append({
        "TYPE": "Reactions",
        "AMOUNT": user_reactions
    })

    user_data.append({
        "TYPE": "Commits",
        "AMOUNT": user_commits
    })

    df_user_data = pd.DataFrame(user_data)
    fig = create_selected_user_barchart(df_user_data, user, repo_url)
    

widgets.interact(Dropdown_Menu, value=user_dropdown) 